# A (Possible) Code for Calibrating the Tagger Timing

First, we import the tools we'll need:

In [1]:
import Acqu as aq
import AcquDetector as aqdet
import ROOT
import numpy as np
from rootpy.plotting import histogram, Hist2D, Hist, Canvas

Welcome to JupyROOT 6.16/00


Import the data to be analyzed:

In [2]:
inFile = '/scratch/2019-05_Timepix/Timepix_33.dat'
aq.openFile(inFile)

Mk2 Data


Load in the detector file for the Tagger:

In [3]:
aqdet.LoadDetectors(['taggerNew.json'])

taggerNew.json


Now we plot the data for the tagger channels and apply a gaussian fit (with an offset) to them (note that onl tbe first 10 channels are done, as it's very slow running to do all the channels at once):

In [28]:
taggerChannels = aqdet.Channels['Tagger'] # number of channels in the tagger (368)

myFit = ROOT.TF1("myFit","gaus(0)+pol0(3)")
myFit.SetParameter(1,880)
myFit.SetParameter(2,10)
myFit.SetParNames("constant","mean","stdev","offset")

histos = [None]*10
offsets = [None]*10
means = [None]*10

for i in range(10):
    histos[i] = Hist(1200,300,1500)

def plotCalTagger():
    data = aqdet.Calibrate(aq.adcArray)
    taggerTimes = aqdet.Arrays['Tagger']['Time']
    
    for dat in taggerTimes:
        for time in dat['Time']:
            for i in range(10):
                if (dat['channel']==(i)):
                    histos[i].Fill(time)  
    
aq.runFunction(plotCalTagger,0,100000)

ROOT.enableJSVis()

c1 = ROOT.TCanvas("my1","Title",1000,2500)
c1.Divide(2,5)

for i in range(10):
    c1.cd(i+1)
    histos[i].SetTitle("Channel "+str(i))
    histos[i].Draw("colz")
    histos[i].Fit('myFit')
    offsets[i] = histos[i].Fit('myFit','SQ').Get().Parameter(3)
    means[i] = histos[i].Fit('myFit','SQ').Get().Parameter(1)
c1.Draw()

[15.842797362417842, 16.086079321434088, 15.66379590393953, 15.926379808053442, 14.494460022706587, 13.036078720040019, 12.29329910517715, 12.61702330062711, 8.382010377576401, 8.354992472754626]
[880.4195923375171, 879.7200582280225, 880.7091592526442, 879.4947059744306, 881.0016522427258, 880.6294672448628, 880.5091957129246, 879.9886782174136, 880.6464083896826, 879.8390560491185]
 FCN=1112.81 FROM MIGRAD    STATUS=CONVERGED     255 CALLS         256 TOTAL
                     EDM=1.56246e-06    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  constant     4.84875e+01   2.14651e+00   2.97212e-02  -5.46983e-04
   2  mean         8.80419e+02   4.18637e-01   6.78490e-03  -1.64704e-03
   3  stdev        8.98590e+00   3.18427e-01   4.37407e-03  -3.05239e-04
   4  offset       1.58427e+01   1.27440e-01   2.05147e-03  -1.00111e-02
 FCN=1098.99 FROM MI

Removing background and setting peaks to 800?:

In [30]:
bins = 1200
offsets2 = [None]*10
ROOT.enableJSVis()
c2 = ROOT.TCanvas("my2","Title",1000,2500)
c2.Divide(2,5)

histos2 = [None]*10
diff = [None]*10
for i in range(10):
    histos2[i] = Hist(bins,300,1500)
    diff[i] = int(round(means[i]-800))

for i in range(10):
    for j in range(bins):
        if (histos[i].GetBinContent(j) - offsets[i] >= 0):
            for k in range(int(histos[i].GetBinContent(j) - offsets[i])):
                histos2[i].Fill(j+300)
        histos2[i].SetBinContent(j-diff[i],histos2[i].GetBinContent(j))
                 
    c2.cd(i+1)
    histos2[i].SetTitle("Channel "+str(i))
    histos2[i].Draw("colz")
    myFit.SetParameter(1,800)
    myFit.SetParameter(2,5)
    histos2[i].Fit('myFit')
    
c2.Draw()

 FCN=2116.43 FROM MIGRAD    STATUS=CONVERGED     131 CALLS         132 TOTAL
                     EDM=1.60888e-07    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  constant     4.44745e+01   5.87332e-01   1.14504e-02   9.65271e-04
   2  mean         8.02404e+02   1.97847e-01   3.94582e-03  -1.40860e-03
   3  stdev        8.94571e+00   1.59068e-01   2.86422e-03   2.43431e-04
   4  offset       2.00018e+00   7.12817e-02   1.56790e-03  -9.93754e-04
 FCN=1761.82 FROM MIGRAD    STATUS=CONVERGED     108 CALLS         109 TOTAL
                     EDM=1.36912e-08    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  constant     3.56476e+01   6.36852e-01   1.16041e-02   2.90993e-04
   2  mean         7.99100e+02   

Testing Zone:

In [41]:
taggerChannels = aqdet.Channels['Tagger'] # number of channels in the tagger (368)

bins = 1200
h1 = Hist(bins,300,1500)

def plotCalTagger():
    data = aqdet.Calibrate(aq.adcArray)
    taggerTimes = aqdet.Arrays['Tagger']['Time']
    
    for dat in taggerTimes:
        for time in dat['Time']:
            if (dat['channel']==1):
                h1.Fill(time)
        
    
aq.runFunction(plotCalTagger,0,50000)

ROOT.enableJSVis()

c1 = ROOT.TCanvas("my1","Title",1000,1000)

h1.SetTitle("Channel 1")
h1.GetXaxis().SetTitle("Time")
h1.Draw("colz")
myFit = ROOT.TF1("myFit","gaus(0)+pol0(3)")
myFit.SetParameter(1,880)
myFit.SetParameter(2,10)
h1.Fit('myFit')


con = h1.Fit('myFit', 'SQ').Get().Parameter(0)
mean = h1.Fit('myFit', 'SQ').Get().Parameter(1)
stdev = h1.Fit('myFit','SQ').Get().Parameter(2)
offset = h1.Fit('myFit','SQ').Get().Parameter(3)
print(con,mean,stdev,offset)

c1.Draw()

(18.782896336644132, 879.1420867213186, 8.825971735291892, 7.266982750767076)
 FCN=1256.07 FROM MIGRAD    STATUS=CONVERGED     266 CALLS         267 TOTAL
                     EDM=2.3061e-08    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           1.87829e+01   1.39515e+00   2.04020e-02   4.76754e-05
   2  p1           8.79142e+02   6.89720e-01   1.19156e-02  -7.73695e-05
   3  p2           8.82595e+00   5.41234e-01   7.87004e-03  -4.16183e-05
   4  p3           7.26699e+00   8.62581e-02   1.47449e-03   2.21378e-03


In [13]:
# write this in for every bin i guess

bins = 1200
ROOT.enableJSVis()
c2 = ROOT.TCanvas("my2","Title",1000,1000)

diff = int(mean - 800)
print(diff)

h2 = Hist(bins,300,1500)

for i in range(bins):
    if (h1.GetBinContent(i) - offset >= 0):
        for j in range(int(h1.GetBinContent(i) - offset)):
            h2.Fill(i+300)
    #print(i+300,int(i-diff)+300,h1.GetBinContent(i))
    h2.SetBinContent(i-diff,h2.GetBinContent(i))

    
h2.Draw()
myFit.SetParameter(1,800)
myFit.SetParameter(2,10)
offset2 = h2.Fit('myFit','SQ').Get().Parameter(3)


#h3 = Hist(bins,300,1500)

#for i in range(bins):
#    if (h2.GetBinContent(i) - offset >= 0):
#        for j in range(int(h2.GetBinContent(i) - offset)):
#            h3.Fill(i+300)


#h3.SetTitle("Channel 1")
#c2.cd(2)
#h3.Draw()
#myFit.SetParameter(1,880)
#myFit.SetParameter(2,5)
#h3.Fit('myFit','','',850,900)
#offset3 = h3.Fit('myFit','SQ').Get().Parameter(3)
    
c2.Draw()



79
